Краткое описание действий функции:

Проверяется, что размерности входного тензора и ядра корректны.
Вычисляются размеры выходного тензора после свертки с учетом параметров stride и padding.
Добавляется паддинг к входному тензору в соответствии с указанным padding.
Инициализируется выходной тензор нулями.
Происходит итерация по батчам, выходным каналам, высоте и ширине выходного тензора.
Для каждого пикселя выходного тензора вычисляется свертка, умножая соответствующие элементы входного тензора и ядра и суммируя результат.
Если указан параметр bias, добавляется соответствующее значение к выходу свертки для каждого выходного канала.
Возвращается полученный выходной тензор.
Таким образом, функция реализует базовую операцию свертки с учетом параметров, которые могут быть настроены пользователем, таких как шаг (stride) и заполнение (padding).

In [1]:
import numpy as np

def convolution2D(input_tensor, weight, bias=None, stride=1, padding=0):
    if len(input_tensor.shape) == 4 and len(weight.shape) == 4:
        batch_size, in_channels, input_height, input_width = input_tensor.shape
        out_channels, _, kernel_height, kernel_width = weight.shape
    else:
        raise ValueError("Неверные размеры входного тензора или ядра")

    # Проверяем, что число входных каналов совпадает
    if in_channels != weight.shape[1]:
        raise ValueError("Число входных каналов в тензоре и ядре должно совпадать")

    # Вычисляем размер выходного тензора после свертки
    output_height = (input_height + 2 * padding - kernel_height) // stride + 1
    output_width = (input_width + 2 * padding - kernel_width) // stride + 1

    # Добавляем паддинг к входному тензору
    padded_input = np.pad(input_tensor, ((0, 0), (0, 0), (padding, padding), (padding, padding)), mode='constant')

    # Инициализируем выходной тензор
    output_tensor = np.zeros((batch_size, out_channels, output_height, output_width))

    # Проходим по батчам, каналам, высоте и ширине выходного тензора
    for b in range(batch_size):
        for o_c in range(out_channels):
            for h_out in range(output_height):
                for w_out in range(output_width):
                    h_start = h_out * stride
                    h_end = h_start + kernel_height
                    w_start = w_out * stride
                    w_end = w_start + kernel_width

                    # Вычисляем свертку для каждого пикселя выходного тензора
                    output_tensor[b, o_c, h_out, w_out] = np.sum(
                        padded_input[b, :, h_start:h_end, w_start:w_end] * weight[o_c, :, :, :]
                    )

    # Добавляем bias, если он предоставлен
    if bias is not None:
        output_tensor += bias.reshape((1, -1, 1, 1))

    return output_tensor

In [5]:
# Пример использования
input_tensor = np.random.rand(2, 3, 4, 4)  # Batch size: 2, Channels: 3, Height: 4, Width: 4
weight = np.random.rand(4, 3, 3, 3)         # Output Channels: 4, Input Channels: 3, Kernel Height: 3, Kernel Width: 3
bias = np.random.rand(4)                     # Bias for each output channel
stride = 1
padding = 1

# Вызываем функцию convolution2D
output_tensor = convolution2D(input_tensor, weight, bias, stride, padding)

# Выводим результаты
print("Input Tensor:")
print(input_tensor)
print("\nWeight (Kernel):")
print(weight)
print("\nBias:")
print(bias)
print("\nOutput Tensor:")
print(output_tensor)

Input Tensor:
[[[[0.23314882 0.39148445 0.71442231 0.9008591 ]
   [0.35922559 0.99981858 0.76363684 0.13631999]
   [0.11733673 0.94417158 0.69435637 0.36697482]
   [0.79169101 0.79016508 0.04840143 0.01889929]]

  [[0.8278443  0.25121743 0.98348976 0.86727951]
   [0.72823026 0.48172071 0.08334341 0.38364972]
   [0.86873102 0.52054249 0.89497642 0.71645055]
   [0.46724311 0.80585002 0.40315365 0.3392572 ]]

  [[0.06145446 0.34545074 0.742483   0.62089049]
   [0.21858446 0.20237001 0.01834002 0.10209789]
   [0.77133211 0.63824574 0.15708099 0.51388021]
   [0.93219716 0.86446862 0.96755992 0.80381241]]]


 [[[0.77198704 0.98402373 0.39424857 0.33101982]
   [0.64195836 0.2930071  0.01149996 0.28734553]
   [0.48284387 0.27306899 0.26791118 0.78881806]
   [0.23779086 0.58277097 0.93527114 0.00804132]]

  [[0.95951784 0.50398577 0.70067087 0.64124223]
   [0.47850008 0.54615476 0.37215577 0.78314499]
   [0.24549923 0.18072547 0.70371832 0.3741829 ]
   [0.04686203 0.29613731 0.11507961 0.211881

In [3]:
import unittest
import numpy as np

class TestConvolution2D(unittest.TestCase):

    def test_result_shape(self):
        # Тест проверяет, что форма выходного тензора правильная
        input_tensor = np.random.rand(2, 3, 4, 4)
        weight = np.random.rand(4, 3, 3, 3)
        bias = np.random.rand(4)
        output_tensor = convolution2D(input_tensor, weight, bias)
        self.assertEqual(output_tensor.shape, (2, 4, 2, 2))  # Пример формы для этого теста

    def test_no_bias(self):
        # Тест проверяет, что результат без bias совпадает с результатом без bias из PyTorch
        input_tensor = np.random.rand(2, 3, 4, 4)
        weight = np.random.rand(4, 3, 3, 3)
        output_tensor_custom = convolution2D(input_tensor, weight)
        
        import torch
        import torch.nn.functional as F
        input_tensor_torch = torch.tensor(input_tensor, dtype=torch.float32)
        weight_torch = torch.tensor(weight, dtype=torch.float32)
        output_tensor_torch = F.conv2d(input_tensor_torch, weight_torch).numpy()

        np.testing.assert_allclose(output_tensor_custom, output_tensor_torch, rtol=1e-5, atol=1e-8)

    def test_padding(self):
        # Тест проверяет, что результат с 'same' padding совпадает с результатом из PyTorch
        input_tensor = np.random.rand(2, 3, 4, 4)
        weight = np.random.rand(4, 3, 3, 3)
        bias = np.random.rand(4)
        output_tensor_custom = convolution2D(input_tensor, weight, bias, padding=1)
        
        import torch
        import torch.nn.functional as F
        input_tensor_torch = torch.tensor(input_tensor, dtype=torch.float32)
        weight_torch = torch.tensor(weight, dtype=torch.float32)
        bias_torch = torch.tensor(bias, dtype=torch.float32)
        output_tensor_torch = F.conv2d(input_tensor_torch, weight_torch, bias=bias_torch, padding=1).numpy()

        np.testing.assert_allclose(output_tensor_custom, output_tensor_torch, rtol=1e-5, atol=1e-8)



In [4]:
unittest.main(argv=[''], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 1.634s

OK
